In [2]:
%load_ext autoreload

In [3]:
%autoreload 2
import sys

sys.path.append("../")
from utils.sequence_utils import generate_random_mutant
from utils.model_architectures import Linear, NLNN, CNNa
from models.Noisy_models.Neural_network_models import NN_model
from models.Ground_truth_oracles.RNA_landscape_models import RNA_landscape_constructor
from models.Noisy_models.Ensemble import Ensemble_models
from evaluators.Evaluator import Evaluator
from models.Ground_truth_oracles.TF_binding_landscape_models import *
from utils.sequence_utils import AAS
from explorers.bo_explorer import BO_Explorer

LANDSCAPE_TYPES_PROTEIN = {"RNA": [], "TF": [], "Protein": ["PF1"], "GFP": []}
bo_explorer_prod = BO_Explorer(virtual_screen=20, alphabet=AAS)
bo_explorer_prod.debug = False
evaluator_bo = Evaluator(
    bo_explorer_prod,
    landscape_types=LANDSCAPE_TYPES_PROTEIN,
    path="../simulations/Protein/eval_BO/",
)
evaluator_bo.evaluate_for_landscapes(evaluator_bo.efficiency, num_starts=3)


loading landscapes RNA: [], TF: [], Protein: ['PF1'], GFP: []
0 TF landscapes loaded.


../models/Ground_truth_oracles/RNA_landscape_models.py:22: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.loaded_landscapes = yaml.load(cfgfile)


PyRosetta-4 2020 [Rosetta PyRosetta4.conda.mac.python37.Release 2020.02+release.22ef835b4a2647af94fcd6421a85720f07eddf12 2020-01-05T17:31:56] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.mac.python37.Release r242 2020.02+release.22ef835b4a2 22ef835b4a2647af94fcd6421a85720f07eddf12 http://www.pyrosetta.org 2020-01-05T17:31:56
core.init: {0} command: PyRosetta -ex1 -ex2aro -database /opt/anaconda3/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=-16405850 seed_offset=0 real_seed=-16405850 thread_index=0
basic.random.init_random_generator: {0} RandomGenerator:init: Normal mode, seed=-16405850 RG_type=mt19937
core.chemical.GlobalResidueTypeSet: {0} Finished initializing fa_standard residue

RuntimeError: Caught an unknown exception!